# Uso de modelos de regresión lineal para predecir las ubicaciones de los pozos petroleros con el mayor margen de beneficio

La empresa minera `OilyGiant` busca construir `200 nuevos pozos petroleros`. Nuestra tarea es encontrar la region con el mayor margen de beneficio para la empresa. Se tiene información sobre las reservas de petróleo en tres regiones específicas. Entrenaremos y probaremos el modelo para cada región en `geo_data_0.csv`. 

`Condiciones:`
- Solo se debe usar la `regresión lineal` para el entrenamiento del modelo`.
- Al explorar la región, se lleva a cabo un `estudio de 500 puntos` con la selección de los `mejores 200 puntos` para el cálculo del beneficio.
- `El presupuesto` para el desarrollo de `200 pozos petroleros` es de `100 millones de dólares`.
- `Un barril de materias primas` genera `4.5 USD de ingresos`. El ingreso de una `unidad de producto es de 4500 dólares` (el volumen de reservas está expresado en miles de barriles).
- Después de la evaluación de riesgo, mantén solo las `regiones con riesgo de pérdidas inferior al 2.5%`. De las que se ajustan a los criterios, se debe seleccionar la región con el beneficio promedio más alto.
- Los datos son sintéticos: los detalles del contrato y las características del pozo no se publican.

Se entrenarán tres modelos, uno para cada región (conjunto de datos). Una vez entrenados, el modelo se aplicará a un conjunto de datos de validación para pronosticar los niveles de reservas de petróleo. Con base en las 200 reservas de petróleo más grandes, se determinará la ganancia potencial utilizando el conjunto de datos de validación. Después de que se hayan procesado y examinado los datos iniciales, se recopilará una distribución de ganancias promedio para cada ubicación. Para cada región, se calcularán y mostrarán la ganancia promedio, el intervalo de confianza del 95% y el riesgo de pérdida. Elegiremos la zona más grande y más ventajosa para que OilyGiant construya su nuevo pozo con base en esas cifras calculadas.

### 1.- IMPORTAMOS LIBRERIAS

In [267]:
# Importar las librerías necesarias

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Biblioteca de importación para formatear moneda
import locale

# Ocultar mensajes de advertencia
import warnings
warnings.filterwarnings('ignore')

### 2.-  CARGA DE LA DATA

A continuación, se leerán los archivos csv `geo_data_0.csv`, `geo_data_1.csv`, `geo_data_2.csv` y se almacenarán en DataFrames.

In [268]:
# Leer y almacenar los archivos csv en DataFrames

df_0 = pd.read_csv('../TT_Project_11/dataset/geo_data_0.csv')
df_1 = pd.read_csv('../TT_Project_11/dataset/geo_data_1.csv')
df_2 = pd.read_csv('../TT_Project_11/dataset/geo_data_2.csv')


### 3.- EXPLORANDO LA DATA

Echemos un vistazo a los datos almacenados en cada uno de los tres DataFrames. Se imprimirán las primeras 15 filas de cada DataFrame, seguidas de un resumen e información general de cada uno de ellos.

#### 3.1. Descripción de datos

- `id`: identificador único de pozo de petróleo
- `f0, f1, f2`: tres características de los puntos (su significado específico no es importante, pero las características en sí son significativas)
- `product`:  volumen de reservas en el pozo de petróleo (miles de barriles).

In [269]:
datasets = [df_0, df_1, df_2]

for i in range(len(datasets)):
    print('df_' + str(i) + ':')
    print(datasets[i].head(15))
    print()

df_0:
       id        f0        f1        f2     product
0   txEyH  0.705745 -0.497823  1.221170  105.280062
1   2acmU  1.334711 -0.340164  4.365080   73.037750
2   409Wp  1.022732  0.151990  1.419926   85.265647
3   iJLyR -0.032172  0.139033  2.978566  168.620776
4   Xdl7t  1.988431  0.155413  4.751769  154.036647
5   wX4Hy  0.969570  0.489775 -0.735383   64.741541
6   tL6pL  0.645075  0.530656  1.780266   49.055285
7   BYPU6 -0.400648  0.808337 -5.624670   72.943292
8   j9Oui  0.643105 -0.551583  2.372141  113.356160
9   OLuZU  2.173381  0.563698  9.441852  127.910945
10  b8WQ6  0.371066 -0.036585  0.009208   70.326617
11  1YYm1  0.015920  1.062729 -0.722248   45.110381
12  zIYPq -0.276476  0.924865  0.095584   89.158678
13  iqTqq  0.212696 -0.111147  5.770095  164.298520
14  Ct5yY -0.018578  0.187516  2.944683  158.633720

df_1:
       id         f0         f1        f2     product
0   kBEdx -15.001348  -8.276000 -0.005876    3.179103
1   62mP7  14.272088  -3.475083  0.999183   26.

#### 3.2. Caracteristicas de datos

Usamos el metodo .info() para obtener las caracteristicas de los datos.

In [270]:
datasets = [df_0, df_1, df_2]

for i in range(len(datasets)):
    print('df_' + str(i) + ':')
    print(datasets[i].info())
    print()

df_0:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

df_1:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

df_2:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 col

Parece que no es necesario realizar ningún procesamiento previo después de revisar las vistas previas y los resúmenes del conjunto de datos. No faltan valores y parece que el tipo de datos de cada columna es correcto. No modificaremos los datos de ninguna manera a medida que avanzamos.

Cabe señalar que los nombres de las columnas no son en absoluto descriptivos. OilyGiant Mining Corporation respondió que, como las características ya se habían preparado para el entrenamiento del modelo, su significado preciso era irrelevante para la tarea en cuestión.

#### 3.3 Pre procesamiento de datos:

Cabe señalar que cada uno de los tres conjuntos de datos contiene una cadena de identificación individual para cada recurso petrolero en la columna `id`. Como los valores de la columna `id` son cadenas, un modelo de regresión lineal no puede utilizarlos. Por lo tanto, es mejor eliminar la columna `id` de cada uno de los tres conjuntos de datos antes de comenzar a entrenar los modelos. El bloque de celdas a continuación eliminará la columna `id` de las tres columnas.

In [271]:
df_0 = df_0.drop('id', axis=1)
df_1 = df_1.drop('id', axis=1)
df_2 = df_2.drop('id', axis=1)

## 4.- MODELAMIENTO

### 4.1 Division de los conjuntos de datos:

Ahora, dividamos los datos, ya que cada conjunto de datos solo contiene valores numéricos. Se creará un conjunto de datos de entrenamiento y un conjunto de datos de validación a partir de cada conjunto de datos. Cada conjunto de datos de entrenamiento se comparará con cada conjunto de datos de validación en una proporción de 3:1. Por lo tanto, el conjunto de datos de validación contendrá el 25 % de los datos totales, mientras que el conjunto de datos de entrenamiento para cada región tendrá el 75 %. El bloque de celdas a continuación utilizará la función `train test split` del módulo `sklearn.model selection` para dividir los datos según las instrucciones.

In [272]:
# Crear los conjuntos de datos de entrenamiento, validación y prueba.
# Los conjuntos de datos se dividirán en un 75% y un 25% para una proporción de 3:1 entre los conjuntos de datos de entrenamiento y validación.

# Inicializando las características de DataFrame para cada región
df_0_features = df_0.drop('product', axis=1)
df_1_features = df_1.drop('product', axis=1)
df_2_features = df_2.drop('product', axis=1)

# Inicializando los objetivos de DataFrame para cada región
df_0_target = df_0['product']
df_1_target = df_1['product']
df_2_target = df_2['product']

# Divida los conjuntos de datos de características y destino para cada DataFrame en conjuntos de datos de entrenamiento y validación
# El tamaño de prueba será 0,25 para que el conjunto de datos de validación contenga el 25 % de los datos.
# y el conjunto de datos de entrenamiento contiene el 75% de los datos
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(df_0_features, df_0_target, test_size=0.25)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(df_1_features, df_1_target, test_size=0.25)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(df_2_features, df_2_target, test_size=0.25)

### 4.5 Entrenamiento del modelo de Regresion lineal

##### Regresión Lineal:
`Objetivo:`
Estimar una relación lineal entre una variable dependiente $𝑦$ y y una o más variables independientes $x_{1}, x_{2}, ..., x_{n}$

`Modelo:`
La fórmula general de la regresión lineal es: 


\begin{align}
y = \beta_{0}\beta_{0} + \beta_{1}x_{1} + \beta_{2}x_{2} + ... +  \beta_{n}x_{n} + e
\end{align}

Donde:
- $𝑦$ es la variable dependiente (continua).
- $\beta_{0}$ es el intercepto.
- $\beta_{1}, \beta_{2}, ..., \beta_{n}$ son los coeficientes.
- $e$ es el término de error.

`Salida:`
La regresión lineal predice valores continuos para $𝑦$

Ahora que los conjuntos de datos de entrenamiento y validación se han dividido para cada conjunto de datos regional, ¡finalmente es hora de entrenar los modelos! Nuevamente, se entrenará un modelo para cada conjunto de datos regional. El modelo de elección será un modelo de regresión lineal, ya que estamos tratando de predecir/calcular un número preciso en un rango continuo. Dado que las posibilidades son mucho más numerosas que solo **0** o **1**, esta no es una tarea de clasificación binaria, por lo que la regresión logística no es un modelo de elección precisa para esta aplicación. Las celdas a continuación inicializarán un modelo de regresión lineal, ajustarán el modelo, predecirán los valores objetivo para el conjunto de datos de validación y luego generarán el volumen promedio de las reservas de petróleo previstas y el error cuadrático medio.

#### 4.5.1 Conjunto de datos 'geo_data_0':

In [273]:
# Crear una instancia de un modelo de regresión logística
model = LinearRegression()

# Ajuste el modelo utilizando los datos de entrenamiento
model.fit(features_train_0, target_train_0)

# Predecir los valores objetivo de las características de validación
predicted_valid_0 = model.predict(features_valid_0)

# Calcular e imprimir el volumen promedio de reservas previstas y modelar RMSE
result = mean_squared_error(target_valid_0, predicted_valid_0)**0.5
print(f"Volumen promedio de reservas previstas: {round(predicted_valid_0.mean(),3)} mil barriles")
print("RMSE del modelo de Regresion Lineal del conjunto de validacion_0:", round(result,3))

Volumen promedio de reservas previstas: 92.706 mil barriles
RMSE del modelo de Regresion Lineal del conjunto de validacion_0: 37.951


La cantidad promedio de reservas de petróleo previstas es de 92 mil barriles. El `error cuadrático medio RMSE` del modelo de regresión lineal es de alrededor de 37. Esto indica que la desviación estándar de los valores esperados podría alcanzar los 37.000 barriles.

#### 4.5.2 Conjunto de datos 'geo_data_1':

In [274]:
# Crear una instancia de un modelo de regresión logística
model = LinearRegression()

# Ajuste el modelo utilizando los datos de entrenamiento
model.fit(features_train_1, target_train_1)

# Predecir los valores objetivo de las características de validación
predicted_valid_1 = model.predict(features_valid_1)

# Calcular e imprimir el volumen promedio de reservas previstas y modelar RMSE
result = mean_squared_error(target_valid_1, predicted_valid_1)**0.5
print(f"Volumen promedio de reservas previstas: {round(predicted_valid_1.mean(),3)} mil barriles")
print("RMSE del modelo de Regresion Lineal del conjunto de validacion_1:", round(result,3))

Volumen promedio de reservas previstas: 68.462 mil barriles
RMSE del modelo de Regresion Lineal del conjunto de validacion_1: 0.89


El `volumen promedio` de las reservas de petróleo estimadas es de `68 mil barriles`. El modelo de regresión lineal tiene un `error cuadrático medio RMSE de aproximadamente 0,8`. Esto significa que los valores previstos pueden variar hasta en 0,9 mil barriles. ¡Ese valor RMSE es extremadamente bajo! Debido a que la variación en los volúmenes de reservas de petróleo es tan pequeña, la gran mayoría de las reservas estarán cerca del promedio de 68.

#### 4.5.3 Conjunto de datos 'geo_data_2':

In [275]:
# Crear una instancia de un modelo de regresión logística
model = LinearRegression()

# Ajuste el modelo utilizando los datos de entrenamiento
model.fit(features_train_2, target_train_2)

# Predecir los valores objetivo de las características de validación
predicted_valid_2 = model.predict(features_valid_2)

# Calcular e imprimir el volumen promedio de reservas previstas y modelar RMSE
result = mean_squared_error(target_valid_2, predicted_valid_2)**0.5
print(f"Volumen promedio de reservas previstas: {round(predicted_valid_2.mean(),3)} mil barriles")
print("RMSE del modelo de Regresion Lineal del conjunto de validacion_2:", round(result,3))

Volumen promedio de reservas previstas: 94.864 mil barriles
RMSE del modelo de Regresion Lineal del conjunto de validacion_2: 40.007


El `volumen promedio` estimado de reservas de petróleo es de `94 mil barriles`. El error cuadrático medio del modelo de regresión lineal es de aproximadamente 40. Esto significa que los valores previstos pueden variar hasta en 40 mil barriles. Este es un valor RMSE bastante grande. De hecho, tiene el RMSE más alto de los tres. Los volúmenes estimados de reservas de petróleo podrían desviarse significativamente del promedio de 94.

#### 4.5.4 Resumen

`Las regiones 0 y 2 tienen un valor RSME alto`, lo que significa que el error es alto por lo que `la exactitud no es buena`, especialmente cuando se `comparan con la región 1`. Esto significa que los `volúmenes de reservas `de petróleo en las regiones 0 y 2 son muy variables, mientras que los de la `región 1 son más predecibles`. Además, el volumen promedio de reservas de petróleo en las regiones 0 y 2 es aproximadamente 1,4 veces el de la región 1. Será interesante observar los resultados del análisis de ganancias para cada región, ya que las regiones 0 y 2 pueden generar ganancias significativamente mayores en función de la distribución de sus reservas de petróleo. La región 1, por otro lado, es más probable que produzca un rango de ganancias más preciso.

## 5.- CALCULO DE BENEFICIOS

#### 5.1. Determinar la producción mínima de petróleo necesaria para alcanzar el punto de equilibrio.

Para comenzar a analizar las ganancias potenciales de cada región, primero calculemos cuántos miles de barriles de petróleo se necesitan para garantizar que la empresa minera OilyGiant no esté construyendo/operando con pérdidas. El bloque de celdas a continuación calculará la cantidad de unidades, o miles de barriles, que el volumen de reserva de petróleo debe proporcionar para que OilyGiant obtenga ganancias. Este cálculo se realiza dividiendo el presupuesto total de OilyGiant (100.000.000 USD) para la construcción de 200 pozos de petróleo por los ingresos generados por una unidad (4.500 USD) de petróleo.

In [276]:
# Inicializar variables constantes
# Los ingresos por unidad son de 4.500 USD (mil barriles)
# El presupuesto para el desarrollo de 200 pozos petroleros es de 100.000.000 dólares.
# Los ingresos de 200 pozos petroleros deben superar los 100.000.000 de dólares
presupuesto = 100000000 # presupuesto 100 millones
pozos_proyectados = 200
ingreso_unidad = 4500 #volumen de reservas está expresado en miles de barriles

# Volumen de reservas necesarias para desarrollar un nuevo pozo sin pérdidas:
sufficient_volume = presupuesto/ingreso_unidad

# Imprima los resultados
print(f"En total se requieren {round(sufficient_volume,3)} mil barriles para evitar construir/operar sin pérdidas.")
print(f"Se requieren aproximadamente {round((sufficient_volume / pozos_proyectados), 3)} mil barriles por reserva.")

En total se requieren 22222.222 mil barriles para evitar construir/operar sin pérdidas.
Se requieren aproximadamente 111.111 mil barriles por reserva.


Con base en los cálculos anteriores, es necesario extraer un mínimo de aproximadamente `22222.222 mil barriles = 22.2 millones de barriles` de petróleo del nuevo pozo para asegurar que OilyGiant no sufra ninguna pérdida en la construcción u operación del pozo. Esto se traduce en un mínimo de alrededor de 111.11 mil barriles de cada reserva en el área. Como es evidente, `los volúmenes de reservas de petróleo promedio proyectados para las tres regiones, como se modela en la Sección 4, son menores a los 111.11 mil barriles por reserva`. Sin embargo, tomando en cuenta el `RMSE de la Región 0 y la Región 2`, es seguro que hay volúmenes de reservas de petróleo que `superan el valor de 111.11`. Desafortunadamente, la `Región 1` tiene menos reservas de petróleo que superan el volumen de 111.11, ya que el volumen de `reserva de petróleo promedio es de alrededor de `68.66 mil barriles`, y el RMSE es de aproximadamente 0.887. Sin embargo, si OilyGiant logra extraer petróleo de los 200 principales pozos petroleros de la Región 1, existe una mayor posibilidad de obtener ganancias en lugar de incurrir en pérdidas al final del año.

#### 5.2. Calcular el beneficio potencial a partir de los valores previstos

Calculemos las ganancias que OilyGiant podría obtener en función de los volúmenes de reservas de petróleo previstos de cada región. Para ello, se desarrollará una función que acepte las predicciones de volumen y el tamaño de muestra de los pozos de petróleo de cada región como argumentos. La función utilizará los 200 pozos de petróleo más grandes de cada región para calcular las ganancias de OilyGiant. Primero, las predicciones de cada región se ordenarán en orden descendente y luego se extraerán los 200 valores más importantes de la serie de predicciones. Estos 200 valores se sumarán para obtener un total, que luego se multiplicará por 4500 USD para obtener los ingresos totales. Finalmente, para obtener la cifra de ganancias final, se restará el presupuesto de OilyGiant del producto del volumen total de reservas y los ingresos por unidad. Además, se mostrará la cantidad objetivo de barriles de petróleo que se deben producir junto con las ganancias que OilyGiant puede anticipar.

In [277]:
# Crear una función para calcular el beneficio a partir de los valores objetivo previstos de cada región

# Función de inicialización: toma 'predicciones' y 'contar' como parámetros
def get_profit(targets, predictions, count):
    # Ordenar los volúmenes previstos del mayor al menor, cantidad de filas `count`
    predictions_sorted = predictions.sort_values(ascending=False)[:count]
    
    # Seleccionar los 200 volúmenes de reserva de petróleo más grandes para cada región, pero utilizar los volúmenes objetivo (volúmenes reales)
    pozos_seleccionados = targets[predictions_sorted.index]
    
    # Calcular la ganancia basada en que una unidad produzca un ingreso de $4,500
    # Restar el presupuesto de $100.000.000 de los ingresos totales
    ganancia = (4500 * pozos_seleccionados.sum()) - presupuesto
    
    # Devolver el valor de la ganancia
    return round(ganancia, 2)


# Ejecute la siguiente función...

# Crear una lista de los valores previstos y los valores reales (objetivos) para cada región
predictions = [pd.Series(predicted_valid_0), pd.Series(predicted_valid_1), pd.Series(predicted_valid_2)]
targets = [target_valid_0.reset_index(drop=True) , target_valid_1.reset_index(drop=True), target_valid_2.reset_index(drop=True)]
profits = []

# Bucle for para ejecutar la función get_profit para los conjuntos de datos previstos de cada región
# Almacenar valores de ganancias en la lista de ganancias
for i in range(len(predictions)):
    profits.append(get_profit(targets[i], predictions[i], 200))

# Establezca la moneda local en USD
locale.setlocale(locale.LC_ALL, '')
    
# Imprima cada valor de ganancia
for i in range(len(profits)):
    print(f"Beneficio de la región {i}: {locale.currency(profits[i], grouping=True)}")
    print(f"Volumen objetivo de reserva de petróleo: {round((profits[i] + 100_000_000) / 4500,2)} mil barriles\n")

Beneficio de la región 0: $36,132,948.89
Volumen objetivo de reserva de petróleo: 30251.77 mil barriles

Beneficio de la región 1: $24,150,866.97
Volumen objetivo de reserva de petróleo: 27589.08 mil barriles

Beneficio de la región 2: $26,266,615.53
Volumen objetivo de reserva de petróleo: 28059.25 mil barriles



El ranking de cada región según sus ganancias calculadas y los volúmenes de reservas de petróleo previstos es la siguiente:

- `1. Región 0` - `Beneficio`: $36 millones, `Volumen`: aprox. 30 millones de barriles
- `2. Región 2` - `Beneficio`: $26 millones, `Volumen`: aprox. 28 millones de barriles
- `3. Región 1` - `Beneficio`: $24 millones, `Volumen`: aprox. 27 millones de barriles

Como se puede observar a partir de las ganancias calculadas anteriormente, la Región 0 generó las mayores ganancias, seguida por la Región 2 y luego la Región 1. Las tres regiones generaron ganancias en lugar de pérdidas. 

`Los volúmenes de reservas de petróleo proyectados son todos superiores a los aproximadamente 22,2 millones de barriles` necesarios para que OilyGiant alcance el punto de equilibrio, por lo que `se proyecta que cada región generará una ganancia para OilyGiant`. A pesar de esperar que la Región 1 produzca menos barriles, parece contener algunas reservas de petróleo muy grandes, ya que produjo una ganancia, requiriendo al menos 200 de ellos. Aunque tanto la Región 1 como la Región 2 generaron ganancias para OilyGiant, `la Región 0 generó la mayor ganancia prevista de más de 36 millones de dólares`. Por lo tanto, basándose únicamente en estas estimaciones, `OilyGiant debería construir su nuevo pozo de petróleo en la Región 0`. Sin embargo, dado que dependemos de las mayores reservas previstas, que pueden no materializarse cuando OilyGiant construya su nuevo pozo, es fundamental examinar las distribuciones de las ganancias promedio para cada región, que se tratarán en la sección siguiente.

## 6.- APLICACION DE METODO BOOTSTRAPPING PARA LA DETERMINACION DE LA REGION MAS RENTABLE

Aunque determinar las ganancias generadas por las predicciones de las principales reservas de petróleo es un punto de partida útil para identificar la región más rentable, no proporciona una imagen completa. Al seleccionar las 200 mayores reservas de petróleo de cada región, obtuvimos la máxima ganancia potencial. Para obtener una comprensión más precisa de la rentabilidad de cada región, construyamos una distribución de valores de ganancia promedio para cada región. Para ello, pasaremos 1000 muestras de 500 pozos de petróleo a la función `get_profit`. La función calculará entonces la ganancia anticipada, como se hizo anteriormente, y guardará el valor de la ganancia en la lista `values`. Una vez que se haya obtenido la distribución, determinaremos la ganancia promedio, el intervalo de confianza del 95% y el riesgo de pérdida para cada región.

In [286]:
# Utilice el método bootstrap con 1000 muestras para encontrar la distribución de ganancias
# Recopilar una distribución de ganancias promedio para cada región en función de las muestras

def get_profit_distribution(targets, predictions, count):
    # Inicializar el estado aleatorio
    state = np.random.RandomState(88888)
    
    # Inicializar la lista de valores
    values = []
    
    # Cree un DataFrame que consta de las series de predicciones y objetivos.
    combined_df = pd.DataFrame()
    combined_df['predictions'] = predictions
    combined_df['targets'] = targets.reset_index(drop=True)
    
    # Bucle for para obtener 1000 muestras y pasarlas a la función get_profit
    for i in range(1000):
        target_subsample = combined_df.sample(n=500, replace=True, random_state=state).reset_index(drop=True)
        values.append(get_profit(target_subsample['targets'], target_subsample['predictions'], count))
    
    # Convierte la lista de valores en una serie de pandas
    values = pd.Series(values)
    
    # Calcular la media de las ganancias
    media_ganancia = values.mean()
    
    # Obtener los límites superior e inferior del intervalo de confianza del 95%
    superior = values.quantile(0.975)
    inferior = values.quantile(0.025)
    
    # Determinar la probabilidad de ganancia (pérdida) negativa
    count = 0
    for value in values:
        if value < 0:
            count += 1
            
    print("Ganancia promedio:", locale.currency(media_ganancia, grouping=True))
    print(f"Volumen objetivo de reserva de petróleo: {round((media_ganancia + 100_000_000) / 4500,2)} mil barriles")
    print("Intervalo de Confianza del 95%:", locale.currency(inferior, grouping=True), 'to', locale.currency(superior, grouping=True))
    print(f"Riesgo de pérdida: {1. * count * 100 / len(values)}%")

for i in range(len(predictions)):
    print('Region ' + str(i))
    get_profit_distribution(targets[i], predictions[i], 200)
    print()

Region 0
Ganancia promedio: $4,481,215.04
Volumen objetivo de reserva de petróleo: 23218.05 mil barriles
Intervalo de Confianza del 95%: $-935,430.13 to $9,297,603.29
Riesgo de pérdida: 4.4%

Region 1
Ganancia promedio: $4,140,035.54
Volumen objetivo de reserva de petróleo: 23142.23 mil barriles
Intervalo de Confianza del 95%: $464,983.91 to $7,901,953.60
Riesgo de pérdida: 1.5%

Region 2
Ganancia promedio: $3,451,073.19
Volumen objetivo de reserva de petróleo: 22989.13 mil barriles
Intervalo de Confianza del 95%: $-1,849,621.28 to $8,548,610.06
Riesgo de pérdida: 11.0%



`Las regiones 0 y 2 superan el umbral de riesgo de pérdida aceptable del 2.5 %,` lo que las hace inadecuadas para el nuevo pozo de OilyGiant. Solo `la región 1 tiene un porcentaje de riesgo de pérdida inferior al 2,5 %`, lo que la convierte en la única región que cumple con los criterios de riesgo y puede considerarse para la ubicación del nuevo pozo de OilyGiant.

`La ganancia promedio obtenida para cada región a través de una distribución de 1000 muestras aleatorias es menor que la ganancia estimada a partir de las 200 reservas de petróleo más grandes de cada región`. Sin embargo, esto era de esperar. La Región 1 y la Región 2 tienen ganancias promedio relativamente altas, y la Región 2 tiene una ganancia promedio ligeramente menor. Por lo tanto, junto con el criterio de riesgo de pérdida, `s`e recomienda que OilyGiant construya su nuevo pozo en la Región 1`, que tiene una de las ganancias promedio más altas.

## 7.- CONCLUSIONES

Iniciamos entrenando el modelo de regresion lineal con los dataset de cada región para `predecir los volumenes promedio de reservas` y el RMSE de cada conjunto de validacion de cada dataset.`Obteniendo que la region 1 geo_data_1` tiene un `modelo mas exacto pero un volumen promedio menor a la region 0 y 2`.

Luego `calculamos las ganancias asociadas a los 200 pozos` con valores predichos mas altos y `se obtuvo que la region 0 (geo_data_0)` obtuvo un `Beneficio`: $36 millones, `Volumen`: aprox. 30 millones de barriles, siendo el beneficio de la region 0 mayor a las regiones 1 y 2, ademas que se superan los volumenes de reservas proyectadas de 22 millones de barriles.  Si bien este fue un buen punto de partida para el análisis, `se necesitaron más muestras para lograr una mayor precisión`. 

Para mejorar la precision de nuestro modelos `se utilizó el método bootstrap para obtener 1000 muestras y determinar la distribución de las ganancias para cada región`. Utilizando `la ganancia promedio resultante`, `el intervalo de confianza del 95%` y los `valores de riesgo de pérdida`, concluyendo que la `Región 1` es la `ubicación más adecuada`, debido a que la region 0 y 2 superan el umbral de riesgo de perdida del 2.5%, ademas que el beneficio inferior y superior del intervalo de confianza es positiva a diferencia de las otras regiones. 

Por lo que concluimos que la `region 1` es la mejor region para para que OilyGiant Mining Company construya su nuevo pozo petrolero, se toma esta decision a pesar de inicialmente se penso que la `region 0` era la de mayor beneficio pero su modelo presentaba alto RMSE, por lo que `luego de la aplicacion del metodo bootstrap la region 1` que mejora la precision del modelo se obtuvo una ganancia promedio mayor y un riesgo de perdida menor al solicitado.




